## Backpropagation

[reference](http://neuralnetworksanddeeplearning.com/chap2.html)

Say cost function is C , the quadratic cost has the form

$$ C = \frac{1}{2n} \sum_x || y(x) - a(x)^L ||^2 $$

where n is the number of training examples. So, cost for single example $C_x = \frac{1}{2} || y- a^L||^2$.

The goal of backpropagation is the partial derivative $\partial C / \partial w$ and $\partial C / \partial b$ of the cost function $C$ with respect to any weight $w$ or bias $b$ in the network


We define the error $\delta_j^l$ of neuron $j$ in layer $l$ by

$$\delta_j^l = \frac{\partial C}{\partial z_j^l}$$

**An Equation for the error in the output layer L**, $\delta^L$: The components of $\delta^L$ are given by

$$ \delta_j^L = \frac{\delta C}{\delta a_j^L} \sigma' (z_j^L) $$

The first term on the right measures how fast the cost is chaning as a function of the $j$-th output activation. Second term on the right measures how fast the activation function $\sigma$ is chaning at $z_j^L$.

It will be computed very easily. If we are using quadratic cost function that we defined above

$$\frac{\partial C}{\partial a_j^L} = (a_j - y_j), $$

So, 

$$\delta^L = (a^L-y) \odot \sigma'(z^L).$$

**An Equation for the error $\delta^l$ in terms of the error in the next layer, $\delta^{l+1}**, 

$$ \delta^l = ((w^{l+1})^T \delta^{l+1})) \odot \sigma'(z^l)$$

**An equation for the rate of change of the cost with respect to any bias in the network**

$$ \frac{\partial C}{\partial b_j^l} = \delta_j^l $$

Because the error is exactly equal to the rate of change $\frac{\partial C}{\partial b_j^l}$,

**An equation for the rate of change of the cost with respect to any weight in the network**

$$ \frac{\partial C}{\partial w_{jk}^l} = a_k^{l-1} \delta_j^l $$

where $k$ is the index in output layer, $j$ is the index in the input layer which is in previous layer.

So, we can re-write 

$$ \frac{\partial C}{\partial w} = a_{in} \delta_{out} $$

* See the code below for better understanding

In [1]:
import numpy as np

class Network():
#...
    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The "mini_batch" is a list of tuples "(x, y)", and "eta"
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
    def backprop(self, x, y):
        """Return a tuple "(nabla_b, nabla_w)" representing the
        gradient for the cost function C_x.  "nabla_b" and
        "nabla_w" are layer-by-layer lists of numpy arrays, similar
        to "self.biases" and "self.weights"."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        
        ## belows for feedforward algorithm
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid_vec(z)
            activations.append(activation)
            
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime_vec(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            spv = sigmoid_prime_vec(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * spv
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

#...

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y) 

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

sigmoid_vec = np.vectorize(sigmoid)

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))
